In [1]:
import chess
import chess.engine
import re
from stockfish import Stockfish
import pandas as pd
import chardet


In [2]:
stockfish = Stockfish()
stockfish.update_engine_parameters({"Threads": 8, "Hash": 4096})

file_path = "/Users/nafis-mac/GitHub Repos/Chess-Research-Project/LLM_vs_LLM-output.txt"

In [3]:
move_pattern = re.compile(r"Qwen (white|black) move:\s*([^\s]+)")

board = chess.Board()
game_number = 0

In [4]:
results = []

In [5]:
with open(file_path, 'rb') as f:
        raw_data = f.read()
        encoding_info = chardet.detect(raw_data)
        encoding = encoding_info['encoding']
        print(f"Detected file encoding: {encoding}")

Detected file encoding: UTF-16


In [6]:
with open(file_path, "r", encoding=encoding) as f:
        for line in f:
            # move_average = 0
            line = line.strip()
            
            # If a new game is starting, reset the board.
            if line.startswith("Starting game"):
                board = chess.Board()
                # stockfish = Stockfish()
                game_number += 1
                move_number = 0
                print(f"\n--- Processing Game Number: {game_number} ---\n")
                continue
            
            # Search for valid move lines
            match = move_pattern.search(line)
            if match:
                move_number += 1
                player = match.group(1)
                move_str = match.group(2)
                move_cps = []
                
                # Get the best move suggestion from Stockfish on the current board state.
                temp_board = board.copy()

                
                stockfish.set_fen_position(temp_board.fen())

                best_move = stockfish.get_best_move()
                temp_board.push_san(best_move)

                stockfish.set_fen_position(temp_board.fen())
                
                for i in range(10):
                    eval = stockfish.get_evaluation()
                    if player == "white" and eval["type"] == "mate":
                         move_cps.append(20000 - eval["value"])
                    elif player == "black" and eval["type"] == "mate":
                         move_cps.append(-20000 - eval["value"])
                    else:
                         move_cps.append(eval["value"])
                best_move_score = sum(move_cps) / len(move_cps)

                move_cps = []

                board.push_san(move_str)
                stockfish.set_fen_position(board.fen())

                for i in range(10):
                    eval = stockfish.get_evaluation()
                    if player == "white" and eval["type"] == "mate":
                         move_cps.append(20000 - eval["value"])
                    elif player == "black" and eval["type"] == "mate":
                         move_cps.append(-20000 - eval["value"])
                    else:
                         move_cps.append(eval["value"])
                qwen_move_score = sum(move_cps) / 10
                # print(move_cps)
                
                print(f"Color: {player} Best: {best_move}  {best_move_score} Actual: {move_str}  {qwen_move_score} Diff: {best_move_score - qwen_move_score} Move: {move_number}")

            results.append({
                "Game Number": game_number,
                "Qwen Color": player,
                "Best Move": best_move,
                "Best Move Score": best_move_score,
                "Qwen Move": move_str,
                "Qwen Move Score": qwen_move_score,
                "Score Difference": best_move_score - qwen_move_score
            })


--- Processing Game Number: 1 ---

Color: white Best: e2e4  30.7 Actual: e4  29.2 Diff: 1.5 Move: 1
Color: black Best: e7e5  30.1 Actual: e5  33.2 Diff: -3.1000000000000014 Move: 2
Color: white Best: g1f3  32.5 Actual: Nf3  30.4 Diff: 2.1000000000000014 Move: 3
Color: black Best: b8c6  30.2 Actual: Nc6  31.7 Diff: -1.5 Move: 4
Color: white Best: f1b5  25.4 Actual: Ng5  -456.2 Diff: 481.59999999999997 Move: 5
Color: black Best: d8g5  -455.0 Actual: Nh6  -27.0 Diff: -428.0 Move: 6
Color: white Best: d2d3  -38.3 Actual: Bb5  -443.0 Diff: 404.7 Move: 7
Color: black Best: d8g5  -442.2 Actual: f6  33.2 Diff: -475.4 Move: 8
Color: white Best: g5f3  28.6 Actual: Bxc6  -68.4 Diff: 97.0 Move: 9
Color: black Best: b7c6  -63.5 Actual: b5  692.3 Diff: -755.8 Move: 10
Color: white Best: g5e6  690.3 Actual: Bxd7+  59.4 Diff: 630.9 Move: 11
Color: black Best: d8d7  59.6 Actual: Bxd7  73.6 Diff: -13.999999999999993 Move: 12
Color: white Best: g5f3  71.6 Actual: Nh3  -21.0 Diff: 92.6 Move: 13
Color: bl

In [7]:
df = pd.DataFrame(results)
df.to_csv("LLM_vs_LLM_ScoreAnalysis.csv", index=False)
print("Results saved to LLM_vs_LLM_ScoreAnalysis.csv")

Results saved to LLM_vs_LLM_ScoreAnalysis.csv


In [9]:
results

[{'Game Number': 1,
  'Qwen Color': 'white',
  'Best Move': 'e2e4',
  'Best Move Score': 30.7,
  'Qwen Move': 'e4',
  'Qwen Move Score': 29.2,
  'Score Difference': 1.5},
 {'Game Number': 1,
  'Qwen Color': 'black',
  'Best Move': 'e7e5',
  'Best Move Score': 30.1,
  'Qwen Move': 'e5',
  'Qwen Move Score': 33.2,
  'Score Difference': -3.1000000000000014},
 {'Game Number': 1,
  'Qwen Color': 'white',
  'Best Move': 'g1f3',
  'Best Move Score': 32.5,
  'Qwen Move': 'Nf3',
  'Qwen Move Score': 30.4,
  'Score Difference': 2.1000000000000014},
 {'Game Number': 1,
  'Qwen Color': 'black',
  'Best Move': 'b8c6',
  'Best Move Score': 30.2,
  'Qwen Move': 'Nc6',
  'Qwen Move Score': 31.7,
  'Score Difference': -1.5},
 {'Game Number': 1,
  'Qwen Color': 'white',
  'Best Move': 'f1b5',
  'Best Move Score': 25.4,
  'Qwen Move': 'Ng5',
  'Qwen Move Score': -456.2,
  'Score Difference': 481.59999999999997},
 {'Game Number': 1,
  'Qwen Color': 'black',
  'Best Move': 'd8g5',
  'Best Move Score': -455

In [22]:
board
stockfish = Stockfish()
stockfish.set_fen_position(board.fen())
stockfish.get_evaluation()
# print(stockfish.get_best_move())

{'type': 'cp', 'value': -473}